# Logistic Regression - Optimization Techniques


The goal of this notebook is to learn how to optimize Logistic Regression based clssification. We will use a large dataset to reveal the benefit of optimization techniques.

We will investigate the following two optimization techniques.

- Optimization Algorithms (solvers)
- Dimensionality Reduction

The dimensionality reduction technique can be generalized to other Machine Learning models.

# Dataset

We will use the MNIST dataset, which is a set of 70,000 small images of digits handwritten by high school students and employees of the US Census Bureau. Each image is labeled with the digit it represents.


There are 70,000 images. Each image is 28x28 pixels, and each feature simply represents one pixel’s intensity, from 0 (white) to 255 (black).

Thus, each image has 784 features. 

In [2]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt


from sklearn.datasets import fetch_mldata
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, classification_report
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler

from sklearn.decomposition import PCA

## Load Data and Create Data Matrix (X) and the Label Vector (y)


In [17]:
mnist = fetch_mldata('MNIST original')


X, y = mnist["data"], mnist["target"] 

print(X.shape)
print(y.shape)

(70000, 784)
(70000,)


/Users/hasan/anaconda/lib/python3.5/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function fetch_mldata is deprecated; fetch_mldata was deprecated in version 0.20 and will be removed in version 0.22
  warnings.warn(msg, category=DeprecationWarning)
/Users/hasan/anaconda/lib/python3.5/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function mldata_filename is deprecated; mldata_filename was deprecated in version 0.20 and will be removed in version 0.22
  warnings.warn(msg, category=DeprecationWarning)


## Split Data Into Training and Test Sets

The MNIST dataset is already split into a training set (the first 60,000 images) and a test set (the last 10,000 images).

We will shuffle the training set to ensure that all cross-validation folds will be similar. 

In [18]:
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]



shuffle_index = np.random.permutation(60000)
X_train, y_train = X_train[shuffle_index], y_train[shuffle_index]

# Optimization using Solvers

Scikit-Learn's LogisticRegression class implements regularized logistic regression using the following algorithms/solvers:

- liblinear library
- newton-cg
- lbfgs
- sag
- saga

These solvers are best understood based on their categories.

Solvers that use **First-order Gradient Descent** Algorithm:
- liblinear
- sag 
- saga

The "sag" and "saga" solvers implement **stochastic gradient descent**, hence they are faster than "liblinear".

Solvers that use **Second-order Gradient Descent** Algorithm:
- newton-cg
- lbfgs

The Second-order Gradient Descent algorithm use the information about the curvature of the space (i.e., the Hessian of the cost function) and thus converge faster. The primary example of this type of algorithm is Newton’s algorithm. Unfortunately, it may be too expensive to compute the Hessian explicitly in Newton's algorithm. 

To remedy this issue, the Quasi-Newton methods are created that iteratively build up an approximation to the Hessian using information gleaned from the gradient vector at each step. The most common method is called BFGS (named after its inventors, Broyden, Fletcher, Goldfarb and Shanno), which updates the approximation. There is a memory efficient version of BFGS wgich is called the limited memory BFGS, or L-BFGS.


We will investigate the performance of these solvers on the MNIST dataset.

Note that the best multi-class classification technique using Logistic Regression is the softmax regression. Only the liblinear solver doesn't support softmax regression. We have to use the one-vs-rest strategy for this solver.


# Optimization using Solvers: Liblinear Library

The solver “liblinear” uses a coordinate descent (CD) algorithm.

The CD algorithm implemented in liblinear cannot learn a true multinomial (multiclass) model. In other words, the liblinear solver does not support softmax regression for multi-class classification.

It uses “one-vs-rest (ovr)” strategy to train separate binary classifiers are trained for all classes. 

## Train the Model Using the liblinear Solver

In [20]:
%%time
log_reg_ovr_liblinear = LogisticRegression(solver='liblinear')

log_reg_ovr_liblinear.fit(X_train, y_train)

/Users/hasan/anaconda/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


CPU times: user 42min 40s, sys: 4.28 s, total: 42min 44s
Wall time: 42min 47s


/Users/hasan/anaconda/lib/python3.5/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


## Evaluate the liblinear Based Model on Test Data

In [21]:
print("No. of Iterations:", log_reg_ovr_liblinear.n_iter_ )


y_test_predicted = log_reg_ovr_liblinear.predict(X_test)
#print(y_test_predict)

accuracy_score_test = np.mean(y_test_predicted == y_test)
print("\nAccuracy: ", accuracy_score_test)


print("\nTest Confusion Matrix:")
print(confusion_matrix(y_test, y_test_predicted))


print("\nClassification Report:")
print(classification_report(y_test, y_test_predicted))

No. of Iterations: [100]

Accuracy:  0.9169

Test Confusion Matrix:
[[ 960    0    2    1    0    3    7    2    3    2]
 [   0 1107    4    3    0    1    5    1   13    1]
 [   9    9  903   20    7    4   13   15   49    3]
 [   4    0   18  918    2   23    5   10   23    7]
 [   1    2    4    5  909    0    9    3   10   39]
 [   9    1    0   39   10  766   16    7   34   10]
 [   8    4    8    0    5   19  907    0    7    0]
 [   3    7   24    6    8    2    1  940    5   32]
 [  12   14    6   21   12   20   10   12  857   10]
 [   8    8    2   14   28    8    0   25   14  902]]

Classification Report:
              precision    recall  f1-score   support

         0.0       0.95      0.98      0.96       980
         1.0       0.96      0.98      0.97      1135
         2.0       0.93      0.88      0.90      1032
         3.0       0.89      0.91      0.90      1010
         4.0       0.93      0.93      0.93       982
         5.0       0.91      0.86      0.88       89

# Optimization using Solvers: SAG

The “sag” solver uses a Stochastic Average Gradient descent. It is faster than other solvers for large datasets, when both the number of samples and the number of features are large.

## Train the Model Using the sag Solver¶

In [27]:
%%time
softmax_reg_sag = LogisticRegression(solver='sag', multi_class='multinomial')

softmax_reg_sag.fit(X_train, y_train)

CPU times: user 3min 15s, sys: 444 ms, total: 3min 16s
Wall time: 3min 16s


/Users/hasan/anaconda/lib/python3.5/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


## Evaluate the sag Based Model on Test Data

In [28]:
print("No. of Iterations:", softmax_reg_sag.n_iter_ )


y_test_predicted = softmax_reg_sag.predict(X_test)
#print(y_test_predict)

accuracy_score_test = np.mean(y_test_predicted == y_test)
print("\nAccuracy: ", accuracy_score_test)


print("\nTest Confusion Matrix:")
print(confusion_matrix(y_test, y_test_predicted))


print("\nClassification Report:")
print(classification_report(y_test, y_test_predicted))

No. of Iterations: [100]

Accuracy:  0.9255

Test Confusion Matrix:
[[ 956    0    1    4    1    5    5    3    4    1]
 [   0 1116    6    2    0    1    3    1    6    0]
 [   5   13  926   16    9    3   11    7   39    3]
 [   4    1   17  925    1   24    3    9   20    6]
 [   1    2    5    4  917    0    9    6   10   28]
 [  11    3    3   39   10  765   15    7   34    5]
 [   9    3    6    3    5   17  913    1    1    0]
 [   1    6   23    9    5    1    0  947    3   33]
 [   8   13    5   22    6   22    8   13  866   11]
 [   8    7    2    8   23    6    0   21   10  924]]

Classification Report:
              precision    recall  f1-score   support

         0.0       0.95      0.98      0.96       980
         1.0       0.96      0.98      0.97      1135
         2.0       0.93      0.90      0.91      1032
         3.0       0.90      0.92      0.91      1010
         4.0       0.94      0.93      0.94       982
         5.0       0.91      0.86      0.88       89

# Optimization using Solvers: SAGA

The “saga” solver is a variant of “sag” that also supports the non-smooth penalty=”l1” option. This is therefore the solver of choice for sparse multinomial logistic regression.

## Train the Model Using the saga Solver¶

In [29]:
%%time
softmax_reg_saga = LogisticRegression(solver='saga', multi_class='multinomial')

softmax_reg_saga.fit(X_train, y_train)

CPU times: user 4min 47s, sys: 447 ms, total: 4min 48s
Wall time: 4min 48s


/Users/hasan/anaconda/lib/python3.5/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


## Evaluate the saga Based Model on Test Data

In [30]:
print("No. of Iterations:", softmax_reg_saga.n_iter_ )


y_test_predicted = softmax_reg_saga.predict(X_test)
#print(y_test_predict)

accuracy_score_test = np.mean(y_test_predicted == y_test)
print("\nAccuracy: ", accuracy_score_test)


print("\nTest Confusion Matrix:")
print(confusion_matrix(y_test, y_test_predicted))


print("\nClassification Report:")
print(classification_report(y_test, y_test_predicted))

No. of Iterations: [100]

Accuracy:  0.9257

Test Confusion Matrix:
[[ 960    0    0    3    1    4    6    3    2    1]
 [   0 1117    5    2    0    1    3    1    6    0]
 [   5   10  925   17    9    3   11   11   39    2]
 [   4    0   17  926    1   22    2    9   22    7]
 [   1    2    5    3  917    0    9    6   10   29]
 [  12    3    2   37   10  767   14    7   35    5]
 [   9    3    7    3    6   17  910    2    1    0]
 [   1    6   23    7    5    1    0  947    4   34]
 [   8   13    5   23    6   23    9   11  864   12]
 [   9    7    1    8   23    7    0   21    9  924]]

Classification Report:
              precision    recall  f1-score   support

         0.0       0.95      0.98      0.97       980
         1.0       0.96      0.98      0.97      1135
         2.0       0.93      0.90      0.91      1032
         3.0       0.90      0.92      0.91      1010
         4.0       0.94      0.93      0.94       982
         5.0       0.91      0.86      0.88       89

# Optimization using Solvers: Newton-CG

The solver “newton-cg” uses implements second order optimization.


## Train the Model Using the newton-cg Solver¶

In [22]:
%%time
softmax_reg_newtoncg = LogisticRegression(solver='newton-cg', multi_class='multinomial')

softmax_reg_newtoncg.fit(X_train, y_train)

CPU times: user 2h 3min 30s, sys: 2min 32s, total: 2h 6min 2s
Wall time: 31min 40s


/Users/hasan/anaconda/lib/python3.5/site-packages/sklearn/utils/optimize.py:203: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  "number of iterations.", ConvergenceWarning)


## Evaluate the newton-cg Based Model on Test Data

In [23]:
print("No. of Iterations:", softmax_reg_newtoncg.n_iter_ )


y_test_predicted = softmax_reg_newtoncg.predict(X_test)
#print(y_test_predict)

accuracy_score_test = np.mean(y_test_predicted == y_test)
print("\nAccuracy: ", accuracy_score_test)


print("\nTest Confusion Matrix:")
print(confusion_matrix(y_test, y_test_predicted))


print("\nClassification Report:")
print(classification_report(y_test, y_test_predicted))

No. of Iterations: [100]

Accuracy:  0.9206

Test Confusion Matrix:
[[ 953    0    6    2    1    6    5    4    3    0]
 [   0 1110    8    3    0    1    3    2    8    0]
 [  13   16  908   20   10    5   11    7   37    5]
 [   4    1   22  920    2   21    2   11   21    6]
 [   3    4    9    5  908    0    8    6    8   31]
 [  10    3    2   35   10  770   15    8   34    5]
 [  10    4   11    2    7   17  906    0    1    0]
 [   1   10   24    7    7    2    0  943    5   29]
 [   8   16    4   22    6   24    9   10  863   12]
 [   6    7    2    9   20    8    1   21   10  925]]

Classification Report:
              precision    recall  f1-score   support

         0.0       0.95      0.97      0.96       980
         1.0       0.95      0.98      0.96      1135
         2.0       0.91      0.88      0.90      1032
         3.0       0.90      0.91      0.90      1010
         4.0       0.94      0.92      0.93       982
         5.0       0.90      0.86      0.88       89

# Optimization using Solvers: L-BFGS

The “lbfgs” is an optimization algorithm that approximates the Broyden–Fletcher–Goldfarb–Shanno (BFGS) algorithm, which belongs to quasi-Newton methods. 

The “lbfgs” solver is recommended for use for small data-sets but for larger datasets its performance might suffer.

## Train the Model Using the lbfgs Solver¶

In [24]:
%%time
softmax_reg_lbfgs = LogisticRegression(solver='lbfgs', multi_class='multinomial')

softmax_reg_lbfgs.fit(X_train, y_train)

CPU times: user 1min 8s, sys: 3.15 s, total: 1min 12s
Wall time: 18.4 s


/Users/hasan/anaconda/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


## Evaluate the lbfgs Based Model on Test Data

In [31]:
print("No. of Iterations:", softmax_reg_lbfgs.n_iter_ )


y_test_predicted = softmax_reg_lbfgs.predict(X_test)
#print(y_test_predict)

accuracy_score_test = np.mean(y_test_predicted == y_test)
print("\nAccuracy: ", accuracy_score_test)


print("\nTest Confusion Matrix:")
print(confusion_matrix(y_test, y_test_predicted))


print("\nClassification Report:")
print(classification_report(y_test, y_test_predicted))

No. of Iterations: [100]

Accuracy:  0.9255

Test Confusion Matrix:
[[ 963    0    0    3    1    3    4    4    2    0]
 [   0 1112    4    2    0    1    3    2   11    0]
 [   3   10  926   15    6    4   15    8   42    3]
 [   4    1   21  916    1   26    3    9   22    7]
 [   1    1    7    3  910    0    9    7   10   34]
 [  11    2    1   33   11  776   11    6   35    6]
 [   9    3    7    3    7   16  910    2    1    0]
 [   1    6   24    5    7    1    0  951    3   30]
 [   8    7    6   23    6   26   10   10  869    9]
 [   9    7    0   11   25    6    0   22    7  922]]

Classification Report:
              precision    recall  f1-score   support

         0.0       0.95      0.98      0.97       980
         1.0       0.97      0.98      0.97      1135
         2.0       0.93      0.90      0.91      1032
         3.0       0.90      0.91      0.91      1010
         4.0       0.93      0.93      0.93       982
         5.0       0.90      0.87      0.89       89

# Summary: Comparison of the Solvers

We observe that the saga solver has the best accuracy.

However, lbfgs is much faster than other solvers and its accuracy is comparable to saga.

Thus, **lbfgs is the best performing solver**.

In [52]:

data = [["liblinear", 0.9169, "42min 47s"], 
        ["sag", 0.9255, "3min 16s"],
        ["saga", 0.9257, "4min 48s"],
        ["newton-cg", 0.9206, "31min 40s"],
        ["lbfgs", 0.9255, "18.4 s"]]
pd.DataFrame(data, columns=["Solver", "Accuracy", "Running-Time"])

,Solver,Accuracy,Running-Time
0,liblinear,0.9169,42min 47s
1,sag,0.9255,3min 16s
2,saga,0.9257,4min 48s
3,newton-cg,0.9206,31min 40s
4,lbfgs,0.9255,18.4 s


# Optimization Using Dimensionaly Reduction

We can optimize the running-time of the Logistic Regression algorithm by reducing the number of features. Our assumption is that the essence or core content of the data does not span along all dimensions. The technique for reducing the dimension of data is known as dimensionality reduction.

For a gentle introduction to various dimensionality reduction technique, see the notebook "Dimensionality Reduction" in the Github repository.

We will use the Principle Component Analysis (PCA) dimensionality reduction technique to project the MNIST dataset (784 features) to a lower dimensional space by retaining maximum variance. 

The goal is to see the improvement in training time due to this dimensionality reduction.

Before we apply the PCA, we need to standardize the data.

## Standardize the Data

PCA is influenced by scale of the data. Thus we need to scale the features of the data before applying PCA. 

For understanding the negative effect of not scaling the data, see the following post:

https://scikit-learn.org/stable/auto_examples/preprocessing/plot_scaling_importance.html#sphx-glr-auto-examples-preprocessing-plot-scaling-importance-py

Note that we fit the scaler on the training set and transform on the training and test set. 

In [38]:
scaler = StandardScaler()

# Fit on training set only.
scaler.fit(X_train)

# Apply transform to both the training set and the test set.
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

## Apply PCA

While applying PCA we can set the number of principle components by the "n_components" attribute. But more importantly, we can use this attribute to determine the % of variance we want to retain in the extracted features.

For example, if we set it to 0.95, sklearn will choose the **minimum number of principal components** such that 95% of the variance is retained.

In [41]:
pca = PCA(n_components=0.95)

pca.fit(X_train)

PCA(copy=True, iterated_power='auto', n_components=0.95, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

## Number of Principle Components

We can find how many components PCA chose after fitting the model by using the following attribute: n_components_

We will see that 95% of the variance amounts to **315 principal components**.

In [53]:
print("Numberof Principle Components: ", pca.n_components_)  

Numberof Principle Components:  315


## Apply the Mapping (Transform) to both the Training Set and the Test Set

In [35]:
X_train = pca.transform(X_train)
X_test = pca.transform(X_test)

## Train The Logistic Regression Model

We use the best performing solver (i.e., lbfgs) to train the model on the PCA transformed data.

In [36]:
%%time
softmax_reg_pca = LogisticRegression(solver='lbfgs', multi_class='multinomial')

softmax_reg_pca.fit(X_train, y_train)

CPU times: user 30.1 s, sys: 1.86 s, total: 32 s
Wall time: 8.04 s


/Users/hasan/anaconda/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


## Evaluate the Model on the Test Data 



In [37]:
print("No. of Iterations:", softmax_reg_pca.n_iter_ )


y_test_predicted = softmax_reg_pca.predict(X_test)
#print(y_test_predict)

accuracy_score_test = np.mean(y_test_predicted == y_test)
print("\nAccuracy: ", accuracy_score_test)


print("\nTest Confusion Matrix:")
print(confusion_matrix(y_test, y_test_predicted))


print("\nClassification Report:")
print(classification_report(y_test, y_test_predicted))

No. of Iterations: [100]

Accuracy:  0.9265

Test Confusion Matrix:
[[ 957    0    1    2    1    6    8    3    2    0]
 [   0 1114    3    2    0    1    3    2   10    0]
 [   7    5  931   17   12    3    9   11   34    3]
 [   3    3   18  919    1   22    3   11   23    7]
 [   1    2    8    2  917    0   10    4    9   29]
 [   7    5    3   33    8  778   13    6   35    4]
 [  12    3    8    2    6   12  912    1    2    0]
 [   0    9   29    5    6    1    0  948    0   30]
 [   6    6    6   22    9   24    7   11  874    9]
 [   9    7    2   10   25    7    0   26    8  915]]

Classification Report:
              precision    recall  f1-score   support

         0.0       0.96      0.98      0.97       980
         1.0       0.97      0.98      0.97      1135
         2.0       0.92      0.90      0.91      1032
         3.0       0.91      0.91      0.91      1010
         4.0       0.93      0.93      0.93       982
         5.0       0.91      0.87      0.89       89

# Summary: Comparison of All Optimization Techniques

We observe that after performing PCA (retaining 95% variance), we achieve
- Best accuracy
- Smallest running-time

Thus, the best way to optimize the Logistic Regression model is to use the best performing solver (it would vary depending on the dataset) and by applying dimensionality reduction.

In [3]:

data = [["liblinear", 0.9169, "42min 47s"], 
        ["sag", 0.9255, "3min 16s"],
        ["saga", 0.9257, "4min 48s"],
        ["newton-cg", 0.9206, "31min 40s"],
        ["lbfgs", 0.9255, "18.4 s"],
        ["PCA+lbfgs", 0.9265, "8.04 s"]]
pd.DataFrame(data, columns=["Solver", "Accuracy", "Running-Time"])

,Solver,Accuracy,Running-Time
0,liblinear,0.9169,42min 47s
1,sag,0.9255,3min 16s
2,saga,0.9257,4min 48s
3,newton-cg,0.9206,31min 40s
4,lbfgs,0.9255,18.4 s
5,PCA+lbfgs,0.9265,8.04 s
